In [44]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import sqlite3
import xgboost as xgb
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report





In [46]:

batch_size = 100
seed = 123
dir="C:/Users/charl/Desktop/Final year Project/GitHubApi/training/functionedDatabase"
train_ds=tf.keras.utils.text_dataset_from_directory(dir, labels='inferred', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds=tf.keras.utils.text_dataset_from_directory(dir, labels='inferred', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

class_names = sorted(os.listdir(dir))
class_dict = {class_name: i for i, class_name in enumerate(class_names)}

train_labels=train_ds.map(lambda x, y: "pos" if y == 1 else "neg")
val_labels = val_ds.map(lambda x, y: "pos" if y == 1 else "neg")

embedding_layer = tf.keras.layers.Embedding(1000, 5)



result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape



# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence. ####changed the vocab size from 10,000 50,000
vocab_size = 50000
sequence_length = 100

Found 16690 files belonging to 2 classes.
Using 13352 files for training.
Found 16690 files belonging to 2 classes.
Using 3338 files for validation.


In [47]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)


embedding_dim=50



In [48]:

# Convert the text data into a feature matrix using the embedding and vectorize layers
train_data = train_ds.map(lambda x, y: (vectorize_layer(x), y)).prefetch(buffer_size=AUTOTUNE)
val_data = val_ds.map(lambda x, y: (vectorize_layer(x), y)).prefetch(buffer_size=AUTOTUNE)

val_text_ds = val_ds.map(lambda x, y: x)
val_features = vectorize_layer(val_text_ds).numpy()


embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
model = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model for a few epochs
history = model.fit(train_data, validation_data=val_data, epochs=10)




ValueError: Exception encountered when calling layer 'text_vectorization_5' (type TextVectorization).

Attempt to convert a value (<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>) to a Tensor.

Call arguments received by layer 'text_vectorization_5' (type TextVectorization):
  • inputs=<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [ ]:
#print("testing")
# Extract the embedding weights from the model and use them to transform the text data into a feature matrix
weights = embedding_layer.get_weights()[0]
weights=np.asarray(weights).astype('float32')
print("testing")

# weights=weights.astype('float32')
# vectorize_layer=vectorize_layer.astype('float32')
#vectorize_layer.set_weights([weights])
text_data = text_ds.batch(batch_size).prefetch(AUTOTUNE)
features = np.array(model.predict(text_data))

# # Train an XGBoost model on the feature matrix
dtrain = xgb.DMatrix(features, label=train_labels)
dval = xgb.DMatrix(val_features, label=val_labels)
params = {'objective': 'binary:logistic', 'eval_metric': 'logloss'}
model = xgb.train(params, dtrain, evals=[(dval, 'validation')], num_boost_round=100)

testing


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).